In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import sys
import os

# Add the parent directory to the Python path
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), "..")))
import src.config as config

In [3]:
from src.inference import get_feature_store

In [4]:
from datetime import  datetime, timedelta
import pandas as pd  

# Get the current datetime64[us, Etc/UTC]  
current_date = pd.Timestamp.now(tz='Etc/UTC')
feature_store = get_feature_store()

# read time-series data from the feature store
fetch_data_to = current_date - timedelta(hours=1)
fetch_data_from = current_date - timedelta(days=1*29)
print(f"Fetching data from {fetch_data_from} to {fetch_data_to}")
feature_view = feature_store.get_feature_view(
    name=config.FEATURE_VIEW_NAME, version=config.FEATURE_VIEW_VERSION
)

ts_data = feature_view.get_batch_data(
    start_time=(fetch_data_from - timedelta(days=1)),
    end_time=(fetch_data_to + timedelta(days=1)),
)
ts_data = ts_data[ts_data.pickup_hour.between(fetch_data_from, fetch_data_to)]

2025-03-03 13:17:08,326 INFO: Initializing external client
2025-03-03 13:17:08,328 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:17:09,576 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214627
Fetching data from 2025-02-02 18:17:08.322378+00:00 to 2025-03-03 17:17:08.322378+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (111.66s) 


In [5]:
ts_data.sort_values(["pickup_location_id", "pickup_hour"]).reset_index(drop=True)

,pickup_hour,pickup_location_id,rides
0,2025-02-02 19:00:00+00:00,2,0
1,2025-02-02 20:00:00+00:00,2,0
2,2025-02-02 21:00:00+00:00,2,0
3,2025-02-02 22:00:00+00:00,2,0
4,2025-02-02 23:00:00+00:00,2,0
...,...,...,...
174440,2025-03-03 13:00:00+00:00,263,102
174441,2025-03-03 14:00:00+00:00,263,119
174442,2025-03-03 15:00:00+00:00,263,119
174443,2025-03-03 16:00:00+00:00,263,89


In [6]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 180468
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype                  
---  ------              --------------   -----                  
 0   pickup_hour         174445 non-null  datetime64[us, Etc/UTC]
 1   pickup_location_id  174445 non-null  int32                  
 2   rides               174445 non-null  int32                  
dtypes: datetime64[us, Etc/UTC](1), int32(2)
memory usage: 4.0 MB


In [7]:
ts_data["pickup_hour"] = ts_data["pickup_hour"].dt.tz_localize(None)

In [8]:
ts_data.info()

<class 'pandas.core.frame.DataFrame'>
Index: 174445 entries, 0 to 180468
Data columns (total 3 columns):
 #   Column              Non-Null Count   Dtype         
---  ------              --------------   -----         
 0   pickup_hour         174445 non-null  datetime64[us]
 1   pickup_location_id  174445 non-null  int32         
 2   rides               174445 non-null  int32         
dtypes: datetime64[us](1), int32(2)
memory usage: 4.0 MB


In [9]:
from src.data_utils import transform_ts_data_info_features
features = transform_ts_data_info_features(ts_data, window_size=24*28, step_size=23)

In [10]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,2,0,1,0,0,0,0,1,1,1,...,0,1,0,1,0,1,1,0,218,2025-03-02 19:00:00
1,3,0,0,0,1,1,0,1,0,2,...,2,3,1,3,3,4,4,3,216,2025-03-02 19:00:00
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,96,2025-03-02 19:00:00
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,57,2025-03-02 19:00:00
4,2,0,0,0,0,0,0,0,0,1,...,0,0,0,0,1,0,1,1,217,2025-03-02 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,2,0,1,0,0,2,0,0,0,...,0,0,0,0,0,0,0,0,26,2025-03-02 19:00:00
247,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,0,0,0,94,2025-03-02 19:00:00
248,0,0,0,0,0,1,0,0,0,1,...,1,0,0,0,0,1,0,0,32,2025-03-02 19:00:00
249,0,3,2,2,0,1,1,0,0,0,...,0,2,1,2,4,4,2,3,168,2025-03-02 19:00:00


In [11]:
from src.inference import load_batch_of_features_from_store
current_date = pd.Timestamp.now(tz='Etc/UTC')
features = load_batch_of_features_from_store(current_date)

2025-03-03 13:19:13,638 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 13:19:13,655 INFO: Initializing external client
2025-03-03 13:19:13,657 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:19:15,120 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214627
Fetching data from 2025-02-02 18:19:13.638871+00:00 to 2025-03-03 17:19:13.638871+00:00
Finished: Reading data from Hopsworks, using Hopsworks Feature Query Service (5.60s) 


In [12]:
current_date

Timestamp('2025-03-03 18:19:13.638871+0000', tz='Etc/UTC')

In [13]:
features

,rides_t-672,rides_t-671,rides_t-670,rides_t-669,rides_t-668,rides_t-667,rides_t-666,rides_t-665,rides_t-664,rides_t-663,...,rides_t-8,rides_t-7,rides_t-6,rides_t-5,rides_t-4,rides_t-3,rides_t-2,rides_t-1,pickup_location_id,pickup_hour
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,2,2025-03-02 19:00:00
1,0,0,0,0,0,0,0,0,0,0,...,1,0,0,0,0,1,0,1,3,2025-03-02 19:00:00
2,1,0,4,1,0,1,0,1,0,1,...,3,5,6,4,2,7,6,8,4,2025-03-02 19:00:00
3,0,0,1,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,6,2025-03-02 19:00:00
4,3,1,2,1,3,2,0,0,0,1,...,1,2,1,2,2,2,4,1,7,2025-03-02 19:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
246,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,259,2025-03-02 19:00:00
247,0,2,0,0,1,1,2,1,0,1,...,2,1,1,1,2,1,2,3,260,2025-03-02 19:00:00
248,25,9,5,14,5,2,0,0,1,0,...,23,38,36,41,58,49,42,35,261,2025-03-02 19:00:00
249,32,24,22,10,7,2,2,2,0,7,...,121,89,87,87,68,72,65,72,262,2025-03-02 19:00:00


In [14]:
from src.inference import load_model_from_registry

model = load_model_from_registry()

2025-03-03 13:19:30,213 INFO: Closing external client and cleaning up certificates.
Connection closed.
2025-03-03 13:19:30,228 INFO: Initializing external client
2025-03-03 13:19:30,230 INFO: Base URL: https://c.app.hopsworks.ai:443
2025-03-03 13:19:31,592 INFO: Python Engine initialized.

Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/1214627

In [15]:
from src.inference import get_model_predictions
predictions = get_model_predictions(model, features)

[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Users\peddi\anaconda3\envs\sp25_taxi\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^


In [16]:
predictions

,pickup_location_id,predicted_demand
0,2,0.0
1,3,0.0
2,4,5.0
3,6,0.0
4,7,2.0
...,...,...
246,259,0.0
247,260,1.0
248,261,22.0
249,262,49.0


In [17]:
predictions.sort_values("predicted_demand", ascending=False).head(10)["pickup_location_id"].values

array([132, 138, 186, 142, 237, 236, 161, 239,  68, 162])